In [1]:
from PyPDF2 import PdfReader
import pandas as pd
import re

In [13]:
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        num_pages = len(pdf_reader.pages)

        text = ''
        for page_number in range(num_pages):
            page = pdf_reader.pages[page_number]
            text += page.extract_text()

        return text



In [18]:
# Provide the path to your PDF file
pdf_file_path = 'week-1a-meet-program.pdf'

# Call the function to extract text from the PDF
extracted_text = extract_text_from_pdf(pdf_file_path)

# Print the extracted text
print(extracted_text)

Tide Swim Team HY-TEK's MEET MANAGER 8.0 - 3:23 PM  6/16/2023  Page 1
Week 1A 2023: Rockville vs. Flower Valley - 6/17/2023
Meet Program - All Events
#1 Boys 12 & Under 100 SC Meter IM
MCSL: 2021 1:03.31 Andrew Vanas
1:09.48 Jack Conger FV Team: 2007
RV Team: 1:04.61 Adriano Arioti2019
RV Pool: 1:04.68 Jason Hernandez-Fuentes2017
ALL* 1:20.00
Age Name   Team Lane Seed Time
Heat   1 of 1   Finals   Starts at 09:00 AM
FV-PV 1:32.62  11 Salvado, J Martin1
RV-PV 1:24.61  10 Zhang, Eric2
FV-PV 1:20.23  12 Stringer, JJ3
RV-PV 1:16.53  12 Guo, Peter4 ALL*
FV-PV 1:27.90  11 Salvado, Nacho5
RV-PV 1:25.93  11 Schwieters, Daniel6
#2 Girls 12 & Under 100 SC Meter IM
MCSL: 2017 1:06.59 Carly Sebring
1:11.84 Sarah Haase FV Team: 2006
RV Team: 1:08.81 Sarah Eliason2018
RV Pool: 1:08.81 Sarah Eliason2018
ALL* 1:21.00
Age Name   Team Lane Seed Time
Heat   1 of 1   Finals   Starts at 09:03 AM
FV-PV 1:19.18  12 Barish, Ayla1 ALL*
RV-PV 1:23.14  10 Young, Sara2
FV-PV 1:15.72  12 Hawman, Sora3 ALL*
RV-PV 1

In [32]:
#assign text
text = extracted_text  # paste your text here
#replace all start with new
text = text.replace('ALL*', '\n')

# Split the text into lines
lines = text.split('\n')

# Prepare a list to hold dictionaries, each representing a swimmer
swimmers = []
current_event = None

for line in lines:
    # Check if the line starts a new event
    if line.startswith('#'):
        current_event = line

    # Check if the line describes a swimmer
    elif re.match(r'^[A-Z]{2}-[A-Z]{2} (\d{1,2}:)?\d{2}\.\d{2}  \d{1,2}', line):
        parts = line.split()

        # Construct a dictionary for the swimmer
        swimmer = {
            'Event': current_event,
            'Team': parts[0][:2],
            'Time': parts[1],
            'Age': parts[2],
            'Name': ' '.join(parts[3:])
        }

        # Append the dictionary to the list
        swimmers.append(swimmer)

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(swimmers)

# Display the DataFrame
print(df)



                                      Event Team     Time Age  \
0        #1 Boys 12 & Under 100 SC Meter IM   FV  1:32.62  11   
1        #1 Boys 12 & Under 100 SC Meter IM   RV  1:24.61  10   
2        #1 Boys 12 & Under 100 SC Meter IM   FV  1:20.23  12   
3        #1 Boys 12 & Under 100 SC Meter IM   RV  1:16.53  12   
4        #1 Boys 12 & Under 100 SC Meter IM   FV  1:27.90  11   
..                                      ...  ...      ...  ..   
249  #48 Girls 15-18 50 SC Meter Butter ϐly   RV    29.90  15   
250  #48 Girls 15-18 50 SC Meter Butter ϐly   FV    34.05  17   
251  #48 Girls 15-18 50 SC Meter Butter ϐly   RV    29.50  17   
252  #48 Girls 15-18 50 SC Meter Butter ϐly   FV    35.31  16   
253  #48 Girls 15-18 50 SC Meter Butter ϐly   RV    30.28  17   

                   Name  
0    Salvado, J Martin1  
1          Zhang, Eric2  
2         Stringer, JJ3  
3           Guo, Peter4  
4       Salvado, Nacho5  
..                  ...  
249        Xiao, Alice2  
250       H

In [33]:
df.head(50)

,Event,Team,Time,Age,Name
0,#1 Boys 12 & Under 100 SC Meter IM,FV,1:32.62,11,"Salvado, J Martin1"
1,#1 Boys 12 & Under 100 SC Meter IM,RV,1:24.61,10,"Zhang, Eric2"
2,#1 Boys 12 & Under 100 SC Meter IM,FV,1:20.23,12,"Stringer, JJ3"
3,#1 Boys 12 & Under 100 SC Meter IM,RV,1:16.53,12,"Guo, Peter4"
4,#1 Boys 12 & Under 100 SC Meter IM,FV,1:27.90,11,"Salvado, Nacho5"
5,#1 Boys 12 & Under 100 SC Meter IM,RV,1:25.93,11,"Schwieters, Daniel6"
6,#2 Girls 12 & Under 100 SC Meter IM,FV,1:19.18,12,"Barish, Ayla1"
7,#2 Girls 12 & Under 100 SC Meter IM,RV,1:23.14,10,"Young, Sara2"
8,#2 Girls 12 & Under 100 SC Meter IM,FV,1:15.72,12,"Hawman, Sora3"
9,#2 Girls 12 & Under 100 SC Meter IM,RV,1:15.73,11,"Diaz, Juli4"


In [35]:
# Function to convert time to seconds for sorting
def time_to_seconds(time):
    if ':' in time:
        minutes, seconds = time.split(':')
        return int(minutes) * 60 + float(seconds)
    else:
        return float(time)

In [ ]:
# Convert time to seconds
df['TimeInSeconds'] = df['Time'].apply(time_to_seconds)

# Sort by event and time
df = df.sort_values(['Event', 'TimeInSeconds'])

# Assign ranks within each event group
df['Rank'] = df.groupby('Event')['TimeInSeconds'].rank()

# Scoring system
scores = {1.0: 6, 2.0: 4, 3.0: 3, 4.0: 2, 5.0: 1}

# Assign scores based on ranks
df['Score'] = df['Rank'].map(scores).fillna(0)

# Drop auxiliary columns
df = df.drop(['TimeInSeconds', 'Rank'], axis=1)

In [36]:
df.to_csv('swimmers.csv', index=False)

In [37]:
# Calculate total points by team
team_scores = df.groupby('Team')['Score'].sum()

# Export the team scores to a CSV file
team_scores.to_csv('team_scores.csv')